# Генерируем тексты для телеграм-бота на LSTM-нейронных сетях

Данный блокнот реализован командой 'Х' (читается как 'Икс' или 'Ха' -- мы сами не определились) в рамках хакатона Высшей Школы Экономики 17 января 2021 года.

Описание постановки задачи и нашего решения можно посмотреть вот тут, начиная с 1:29:00: 

*   [https://youtu.be/u_8Yj9eKtp4?t=5364](https://youtu.be/u_8Yj9eKtp4?t=5364)


В основе решения несколько работ, они являются гораздо более подробно описанными и технически грамотными, чем наше коленочное решение для задачи хакатона, поэтому советую в первую очередь ознакомиться с ними: 


*   [Туториал](https://www.tensorflow.org/tutorials/text/text_generation) от Гугла о текст-генерации -- ОСНОВА
*   [Статья](https://www.researchgate.net/publication/335158858_LSTM_vs_GRU_vs_Bidirectional_RNN_for_script_generation) и тематический [репозиторий](https://github.com/sanidhyamangal/gru_lstm_tv_script) Мангала со сравнением распространенных архитектур нейросети и оценкой качества получившихся текстов на примере датасета со сценариями сериалов. 
*   [Публикация](https://ods.ai/projects/deep2ch) ODS сообщества о генерации текста и тематическое [видео](https://youtu.be/1LcdA0Y7IEk) по ней. Знающие люди порекомендовали почитать, так что я рекомендую тоже ознакомиться. Сам пока что не читал, но одобряю.
*    Для Телеграм-бота мы использовали [вот этот репозиторий](https://github.com/python-telegram-bot/python-telegram-bot) и пример из файла [conversationbot.py](https://github.com/python-telegram-bot/python-telegram-bot/blob/master/examples/conversationbot.py)

Ниже по тексту по-русски -- наши авторские замечания и заметки. По-английски -- оригинальные комментарии умных людей из Google. Возможно имеет смысл всё перевести на английский, но пока что мне лень. 


## Пара замечаний о модели

Хотя некоторые предложения грамматически корректны, большинство из них не имеет смысла. Модель не выучивает значения слов, но:

*    Модель основана на буквах. Когда началось обучение, модель не знала, как написать слово или что слово является единицей текста.

*    Модель обучается на небольших пакетах текста (по 100 символов каждый) и по-прежнему способна генерировать более длинную последовательность текста с согласованной структурой.

*    Выбор именно такой архитектуры модели продиктован статьей Sanidhya Mangal и личным опытом. Шагом вперед было бы оценивать качество разных генераций модели самими читателями и возвращать эти оценки обратно в модель для дообучения, но для меня это пока что overkill.

*    Не забывайте выбрать GPU в настройках блокнота ("Среда выполнения" -> "Изменить среду выполнения"). Так должно обучаться быстрее. Недавно туда добавили ещё и TPU, но я их не пробовал.

## Setup

### Import TensorFlow and other libraries

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time

In [ ]:
print(tensorflow.__version__)

### Загрузим исходный файл с данными для обучения


In [ ]:
# Для удобства я загрузил датасет с бугуртами на Гитхаб и обращаюсь к нему через raw.githubusercontent
# Это самый удобный и массов воспроизводимый способ загружать датасеты текста в гугл-колаб. 
# У него только один минус -- загрузить объемный датасет в ГитХаб может оказаться неудобной задачей. Есть ограничения самого ГитХаба на это. 

# path_to_file = tf.keras.utils.get_file('BT.txt', 'https://raw.githubusercontent.com/Nejel/datasets/master/BT.txt') 

5332992/5328159 [==============================] - 0s 0us/step


In [ ]:
# Альтернативный способ -- загрузить датасет со своего гугл-диска. 
# Для этого надо создать там папку (в моём случае Colab Notebooks и сложить туда файлы с текстом для обучения).
# Пример ниже: 

from google.colab import drive
drive.mount('/content/drive')

text = open('/content/drive/My Drive/Colab Notebooks/res.txt', 'rb').read().decode(encoding='utf-8')


Mounted at /content/drive


In [ ]:
#HealthCheck

# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

print(text[:300])

"","2"
небо, аллах,

"3227","Я раздвоился. И вот жду, сознающий, на опушке, а — другой — совершаю в далеких полях заветное дело. И — ужасный сон! — непостижно начинаю я, ожидающий, тосковать о том, совершающем дело, и о совершенном деле…  
небо, аллах,


небо, аллах,

***
небо, аллах,




In [ ]:
# Заметно, что в датасете очень часто встречается фраза "небо, аллах". 
# Это возникло из-за ошибки в функции генерации датасета.
# К сожалению я слишком поздно заметил это в день хакатона и переобучить новую модель на более правильных данных банально не успел.)
# Дети, не повторяйте моих ошибок!

## Process the text

In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

289 unique characters


### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '#' :   5,
  '$' :   6,
  '%' :   7,
  '&' :   8,
  "'" :   9,
  '(' :  10,
  ')' :  11,
  '*' :  12,
  '+' :  13,
  ',' :  14,
  '-' :  15,
  '.' :  16,
  '/' :  17,
  '0' :  18,
  '1' :  19,
  ...
}


In [ ]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'"","2"\r\nнебо,' ---- characters mapped to int ---- > [  4   4  14   4  20   4   1   0 230 222 218 231  14]


### Задача предсказания следующего символа

Имея последовательность символов, какой наиболее вероятный следующий символ? Это задача, для которой мы будем оптимизировать наш алгоритм. На ввод в модель будет подаваться последовательность символов и модель будет на каждом шаге возвращать следующий за этой последовательностью символ.

### Создание тренировочных примеров и целей

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [ ]:
# The maximum length sentence we want for a single input in characters
seq_length = 200
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

"
"
,
"
2


The `batch` method lets us easily convert these individual characters to sequences of the desired size.

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'"","2"\r\nнебо, аллах,\r\n\r\n"3227","Я раздвоился. И вот жду, сознающий, на опушке, а — другой — совершаю в далеких полях заветное дело. И — ужасный сон! — непостижно начинаю я, ожидающий, тосковать о том, '
'совершающем дело, и о совершенном деле…  \r\nнебо, аллах,\r\n\r\n\r\nнебо, аллах,\r\n\r\n***\r\nнебо, аллах,\r\n\r\n\r\nнебо, аллах,\r\n\r\nХоть и не вышло, а хорошая мысль стихотворения; убийца-двойник — совершит и отпадет, '
'а созерцателю-то, который не принимал участия в убийстве, — вся награда. Мысль-то сумасшедшая, да ведь и награда — сумасшествие, которое застынет в сладостном созерцании совершенного другим. Память о н'
'оже будет идеальна, ибо нож был хоть и реален, но в мечтах — вот она, великая тайна…"\r\nнебо, аллах,\r\n\r\n"3228","Вот в эту минуту я настолько утончен и уравновешен, что смогу завещать. Складываются в одн'
'о местечко (страшная близость яви и сна, земли и неба!) — укромненькое — все бумаги (и эта) и иные книги. Первое место — «Северные цветы» — не за с

For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [ ]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '"","2"\r\nнебо, аллах,\r\n\r\n"3227","Я раздвоился. И вот жду, сознающий, на опушке, а — другой — совершаю в далеких полях заветное дело. И — ужасный сон! — непостижно начинаю я, ожидающий, тосковать о том,'
Target data: '","2"\r\nнебо, аллах,\r\n\r\n"3227","Я раздвоился. И вот жду, сознающий, на опушке, а — другой — совершаю в далеких полях заветное дело. И — ужасный сон! — непостижно начинаю я, ожидающий, тосковать о том, '


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 4 ('"')
  expected output: 4 ('"')
Step    1
  input: 4 ('"')
  expected output: 14 (',')
Step    2
  input: 14 (',')
  expected output: 4 ('"')
Step    3
  input: 4 ('"')
  expected output: 20 ('2')
Step    4
  input: 20 ('2')
  expected output: 4 ('"')


### Создание тренировочных пакетов (batch)

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 200), (64, 200)), types: (tf.int64, tf.int64)>

## Построение модели

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
# #Базовая и самая простая модель для генерации текстов. 
# #Сейчас закомментирована в пользу более эффективных моделей типо 0.03 (см. ячейку ниже)

# def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
#   model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(vocab_size, embedding_dim,
#                               batch_input_shape=[batch_size, None]),
#     tf.keras.layers.GRU(rnn_units,
#                         return_sequences=True,
#                         stateful=True,
#                         recurrent_initializer='glorot_uniform'),
#     tf.keras.layers.Dense(vocab_size)
#   ])
#   return model

In [ ]:
# Возможно лучшая модель для генерации, из тех что я пробовал и из тех что пробовал Mangal.

# Она казалась мне достаточно простой чтобы быстро обучаться, но в датасете с дневниками она начала работать в РАЗЫ дольше, чем на моих предыдущих тестовых датасетах (с бугуртами). 
# Тем не менее, уверен что на 20-25 эпохах и более-менее правильных датасетах может показать классный результат.
# Вопрос только в том, есть ли у вас столько времени и итераций для обучения. 


MODEL_VERSION = 0.03

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    # embedding layer
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    # LSTM layer
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    
    # dropout layer
    tf.keras.layers.Dropout(0.4),
    
    # LSTM layer
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    # dense layer
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

# Билд модели

In [ ]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU (or LSTM) one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_training.png?raw=1)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 200, 289) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           73984     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dropout (Dropout)            (64, None, 1024)          0         
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dense (Dense)                (64, None, 289)           296225    
Total params: 14,009,889
Trainable params: 14,009,889
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [ ]:
sampled_indices

array([ 19,  17,   7, 173, 189,  59,  23,  93, 216, 263, 219, 178, 257,
       228, 190, 183,   0,  49,  36,  37,  25,  41, 266, 101, 211, 109,
       157, 274,  73, 225, 259,  52, 225, 186, 122,   8,  98, 118, 136,
       213,  30,  91, 106,  97, 161, 189, 142, 221, 166,  67, 180, 231,
       273, 164, 123, 270, 274, 226, 153, 200,   1,  47, 202, 158,  49,
       192,   4, 141,  76,  59, 209, 241, 172, 241, 217, 180, 131, 212,
         4, 202,  43, 186, 162,  61,  12, 107, 116, 260, 136, 222, 166,
       172, 238, 190, 218, 207, 123, 265, 263,  87, 159,  89, 247, 203,
       284,   4, 104,  82, 208,  68, 243, 128, 117, 159,  34, 156, 131,
       258, 237,   6, 242,   0, 193,  67, 213,  16, 223, 116,  82, 218,
        19,  82, 237,  38, 115, 145, 198, 177,  36, 274, 233,  85, 208,
       167, 193,  49, 211, 124, 197, 238, 103,  92, 166,  70, 263, 111,
        81, 174,  92,  57, 136,  96, 220, 148,  76,  10,  83, 123, 193,
       275, 273, 277,  44, 276, 209, 153, 187, 205, 268, 170, 20

Decode these to see the text predicted by this untrained model:

## Посмотрим, что возвращает модель ещё не будучи обученной

In [ ]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'о и неохотно.  \r\nнебо, аллах,\r\n\r\n\r\nнебо, аллах,\r\n\r\nПрошел слух о скором отъезде наместника.  \r\nнебо, аллах,\r\n\r\n\r\nнебо, аллах,\r\n\r\nБегство жителей из крепости продолжается.  \r\nнебо, аллах,\r\n\r\n\r\nнебо, ал'

Next Char Predictions: 
 '1/%σДZ5|ЯႵвψѣлЕЅ\nPCD7H‑¬Ъ¼γ„hиѳSиБç&¥àüЬ<z¸£ηД˃дμbύо”κè‘„йΣП\rNСδPЗ"˂kZШшςшаύôЫ"СJБθ\\*ºÜѵüеμςхЕбЦèὸႵvεxюТ♂"±qЧcъïßεAβôѲф$щ\nИbЬ.жÜqб1qфE×́НχC„рtЧνИPЪéМх°{μeႵ¾pτ{Xü\xa0гΕk(rèИ†”›K…ШΣВФ—πШΚНюѲmѣ!б/-Ч&\xadz‘⅓є£A'


## Обучим получившуюся модель

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Добавим функцию оптимизации (optimizer) и функцию потерь (loss function)

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 200, 289)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.666701


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Выполнение обучения


To keep training time reasonable, use 10 to 30  epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [ ]:
EPOCHS = 3

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/3
2970/2970 [==============================] - 1241s 417ms/step - loss: 1.8866
Epoch 2/3
2970/2970 [==============================] - 1250s 420ms/step - loss: 1.2445
Epoch 3/3
2970/2970 [==============================] - 1264s 425ms/step - loss: 1.1930


## Теперь, когда модель обучена, можно попробовать сгенерировать новый текст

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#ls drive/
# !ls "/content/drive/My Drive"

In [ ]:
# Надо попробовать засейвить модель, чтобы её можно было локально открыть
# https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_load.ipynb#scrollTo=sI1YvCDFzpl3

model.save('/content/drive/My Drive/New_Model')

# model.save('/content/drive/My Drive')
# После этого модель окажется сохранена на гугл-драйве в двух папках: 
# assets (папка остается пустой в нашем случае)
# variables (два файла, один из них на несколько десятков мегабайт)
# также сам файл модели на уровень выше, не потеряйте его. 

# файлы из второй папки забираем и передаем товарищу, либо запускаем локально при помощи функции 
# tf.keras.models.load_model()

INFO:tensorflow:Assets written to: /content/drive/My Drive/New_Model/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/New_Model/assets


In [ ]:
# Загрузим нашу модель с гугл-диска
from google.colab import drive
drive.mount('/content/drive')

new_model = tf.keras.models.load_model('/content/drive/My Drive/New_Model')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Попробуем:
print(generate_text(new_model, temperature, start_string=u"И ТУТ ТеТЯ ЙОБА ГОВОРИТ"))
# Работает!

И ТУТ ТеТЯ ЙОБА ГОВОРИТА    
небо, аллах,


небо, аллах,

8) Получил сегодня письмо от Бооля, который мне сказал, что он предложил мне предполагать представить им деньги и просил меня принять его на все постановки и приглашенный в Училище его завтра в составе приезда


### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.


![To generate text the model's output is fed back to the input](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_sampling.png?raw=1)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
temperature = 0.47

def generate_text(model, temperature, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 250

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = temperature

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))



### Вызовем функцию генерации текста


In [ ]:
print(generate_text(model, temperature, start_string=u"И ТУТ ТЁТЯ ЙОБА ГОВОРИТ: "))

## Теперь попробуем генерировать тексты массово

In [ ]:
count_of_burgers = 5 #count will be miltiplied by count of lines in bugstarts

bugstarts = ('И ТУТ ТЁТЯ ЙОБА ГОВОРИТ: ', 
             'ВСТРЕЧАЕШЬСЯ С ТЯН ', 
             'АНИМЕ НА АВЕ ', 
             'СМОТРИШЬ НОВОСТИ О КОРОНАВИРУСЕ ', 
             'ЕОТОВА ЗНАКОМИТСЯ С ТВОИМ БАТЕЙ ')

for i in range (count_of_burgers):
  print(i+1, " out of ", count_of_burgers)
  for i in range(len(bugstarts)):
    results = open('/content/drive/My Drive/Colab Notebooks/Results.txt', 'a')
    lines = (generate_text(model, temperature, start_string=bugstarts[i]), "\n\n", "temperature: ", str(temperature), "  EPOCHS: ", str(EPOCHS), 
             " MODEL_VERSION: ", str(MODEL_VERSION), " BATCH_SIZE: ", str(BATCH_SIZE), 
             " BUFFER_SIZE: ", str(BUFFER_SIZE), " embedding_dim: ", str(embedding_dim), 
             " rnn_units: ", str(rnn_units), " seq_length: ", str(seq_length), "\n\n\n\n\n")
    results.writelines(lines)
    results.close()

The easiest thing you can do to improve the results it to train it for longer (try `EPOCHS=30`).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.